In [1]:
# (C) Copyright Aaron Goldberg, 2023.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [2]:
# Implementing idea from https://doi.org/10.1103/PhysRevLett.128.160501. Let's use n=2, amplify a squeezed state, and vary the gain coefficient by varying the beam splitter transmissivity. Other n will take longer
import strawberryfields as sf
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock
from strawberryfields.tdm import borealis_gbs, full_compile, get_mode_indices
import numpy as np

2023-05-23 19:53:44.617224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Compilation will make sure the specs match an actual device, so must be done using a particular device and its characteristics
eng = sf.RemoteEngine("borealis")
device = eng.device

In [4]:
print(device.certificate) # Tells us the loss parameters, squeezing parameters

{'finished_at': '2023-05-23T17:41:43.088074+00:00', 'target': 'borealis', 'loop_phases': [1.279, 0.149, 1.976], 'schmidt_number': 1.147, 'common_efficiency': 0.384, 'loop_efficiencies': [0.875, 0.883, 0.793], 'squeezing_parameters_mean': {'low': 0.688, 'high': 1.142, 'medium': 0.969}, 'relative_channel_efficiencies': [0.932, 0.937, 0.922, 0.998, 0.966, 0.93, 0.899, 0.974, 0.96, 0.959, 0.978, 1.0, 0.946, 0.973, 0.953, 0.905]}


In [5]:
# Let's define our circuit manually
modes = 22
starting_mode = 0 # This can be modified if we are aiming for some particular detectors that are better than the rest)

squeeze_level = "high"

# squeezing-gate parameters. These will be rounded to the closest one supported by hardware
r = [0.] * modes

# rotation-gate parameters. These phases will not matter for the photon counting multiplexing we are doing here
phi_0 = [0.] * modes
phi_1 = [0.] * modes
phi_2 = [0.] * modes


# beamsplitter parameters. Set irrelevant transmission parameters to 1 to load all loops
T_0 = [1.] * modes
T_1 = [1.] * modes
T_2 = [1.] * modes


# Create a two-mode squeezed vacuum in the first two modes by interfering two single-mode squeezed vacuum states
r[0+starting_mode] = device.certificate["squeezing_parameters_mean"][squeeze_level] 
r[1+starting_mode] = device.certificate["squeezing_parameters_mean"][squeeze_level]

# symmetric BS will convert two SV into one TMSV
T_0[1+starting_mode] = 0.5 


# Will postselect mode 0 to be in state |n> to herald mode 1 as starting in state |n>

# Create a state to be amplified; here it's a single-mode squeezed vacuum
r[2+starting_mode] = device.certificate["squeezing_parameters_mean"][squeeze_level]
# Delay the state psi in mode 2 to be amplified (all of the squeezed states needed to be in the first modes)
T_0[2+starting_mode] = 1.
delay_length = 11
T_0[2+starting_mode+1:2+starting_mode+1+delay_length] = [0. for i in range(delay_length)]
T_0[2+starting_mode+1+delay_length+1] = 1. # Just to make sure it gets out


# Fixing phases
#phi_0[8]= np.pi/2
#phi_1[1] = np.pi
phi_0[1+starting_mode+1:2+starting_mode+1+delay_length] = [-np.pi/2 for i in range(delay_length+1)] # The delays add phases too, even though they're just identity transformations



# Amplification starts by a variable-transmissivity beam splitter between a vacuum state in mode 7 and the state |n> in mode 1. Will measure mode 1 as amplified state
gain = 0.25
tau = (1./gain)**2/(1.+(1./gain)**2)
tau = 1./(1.+gain**2)
print("tau is", tau)
# We actually want the mode 1 to house the amplified state, which requires an extra swap relative to the paper
# So we adjust the BS accordingly. This does a1-> sqrt(1-tau)a1-sqrt(tau)a2
phi_1[1+starting_mode] += np.pi/2 # To fix the default phases from the beam splitter
T_1[7+starting_mode] = 1.-tau 
phi_2[1+starting_mode] += -np.pi/2 # To fix the default phases from the beam splitter



# Next do QFT on modes 7, 2 delayed to 13, and 14. Then postselect on pattern 0,1,1 to get amplified state, but also will get the same state with different phases if postselect on 1,1,0 or 1,0,1, so that's good too.
# The first BS does a13 -> sqrt(1/2)a13-sqrt(1/2)a14. Our phase choice will end up just adding a global phase
T_0[14+starting_mode] = 0.5
phi_0[2+starting_mode] += -np.pi/2 # Fix the phase. Since this must happen before going into the loop, that happened way back in time bin 2, which then remained in the loop for a long time
phi_1[13+starting_mode] += np.pi/2 # Fix phase after exiting loop.

phi_1[7]+=np.pi
#phi_1[13]+=np.pi/2
#phi_0[2+starting_mode] += np.pi/2


# Now delay mode 14 an appropriate length so that it can intefere at the second loop
delay_length2 = 5
T_0[14+starting_mode+1:14+starting_mode+1+delay_length2] = [0. for i in range(delay_length2)]
T_0[14+starting_mode+1+delay_length2+1] = 1. # Just to make sure it gets out
phi_0[14+starting_mode+1:14+starting_mode+1+delay_length2] = [-np.pi/2 for i in range(delay_length2)]
phi_1[19]+=-np.pi/2

phi_0[2]+=-np.pi/2
phi_0[8]+=-np.pi/2
phi_0[14]+=-np.pi/2
phi_2[7]+=np.pi/2
phi_2[13]+=np.pi/2
phi_2[19]+=-np.pi/2
# Beam splitter between modes 7 and 13 at second loop
phi_1[7+starting_mode] += np.pi/2 
T_1[13+starting_mode] = 1./3  ### this one is not affecting anything yet
phi_2[7+starting_mode] += -np.pi/2

# Put phase on mode 14 that is now in mode 19
phi_1[19]+=np.pi*3./2.




# BS between modes 13 and 19 at second loop
phi_1[19+starting_mode] += np.pi/2 # This will cancel with the one from before, anyway 
T_1[19+starting_mode] = 0.5
phi_2[19+starting_mode] += -np.pi/2

# Finally, postselect on mode 19 having a 1, mode 13 having a 1, mode 7 having a 0. Already said to postselect on n=2 in mode 0. 
# amplified state is in mode 1, so we will look at the (conditional) photon-number distribution there



alpha_0 = np.arccos(np.sqrt(T_0))
alpha_1 = np.arccos(np.sqrt(T_1))
alpha_2 = np.arccos(np.sqrt(T_2))

# the travel time per delay line in time bins
delay_0, delay_1, delay_2 = 1, 6, 36

# set the first beamsplitter arguments to 'T=1' ('alpha=0') to fill the
# loops with pulses
alpha_0[:delay_0] = 0.0
alpha_1[:delay_1] = 0.0
alpha_2[:delay_2] = 0.0

# The gate arguments need to be defined as lists, so if they were defined with numpy we need to cast them to lists
gate_args = {
    "Sgate": r,
    "loops": {
        0: {"Rgate": phi_0, "BSgate": alpha_0.tolist()},
        1: {"Rgate": phi_1, "BSgate": alpha_1.tolist()},
        2: {"Rgate": phi_2, "BSgate": alpha_2.tolist()},
    },
}

tau is 0.9411764705882353


In [6]:
# Now compile:
gate_args_list = full_compile(gate_args, device)

In [7]:
# Verify components if desired
#print(gate_args_list[0])

In [8]:
delays = [1, 6, 36]
vac_modes = sum(delays)

n, N = get_mode_indices(delays)
print(vac_modes)
print(n,N)

43
[43 42 36  0] 44


In [9]:
# Use this if want to manually add realistic losses gleaned from the device characteristics, can then be tweaked
eta_glob = device.certificate["common_efficiency"]
etas_loop = device.certificate["loop_efficiencies"]
etas_ch_rel = device.certificate["relative_channel_efficiencies"]

prog_length = len(gate_args_list[0])
reps = int(np.ceil(prog_length / 16))
etas_ch_rel = np.tile(etas_ch_rel, reps)[:prog_length]
etas_ch_rel = etas_ch_rel.tolist()

gate_args_list += [etas_ch_rel]

from strawberryfields.ops import LossChannel
print("minimum loss for any mode is: ", 1-eta_glob)

minimum loss for any mode is:  0.616


In [10]:
prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
    MeasureFock() | q[0]

In [11]:
shots = 1_000_000
job = eng.run_async(prog, shots=shots, crop=True)
print(job)

<Job: id=eb84a20f-0fbd-4da0-8569-0c12c424de08>


In [ ]:
job.wait()
print(job.status)
my_samp=np.reshape(sf.Result(job.result).samples[:,:,[vac_modes+0,vac_modes+1,vac_modes+7,vac_modes+13,vac_modes+19]],(shots,5))
np.savetxt(f'Borealis_teleamp_raws_{gain}_v2.csv', my_samp.astype(int), delimiter=',',fmt='%i')

In [ ]:
unique, counts = np.unique(my_samp, return_counts=True,axis=0) 
np.savetxt(f'Borealis_teleamp_patterns_{gain}_v2.csv', unique.astype(int), delimiter=',',fmt='%i')
np.savetxt(f'Borealis_teleamp_counts_{gain}_v2.csv', counts.astype(int), delimiter=',',fmt='%i')

In [ ]:
# Little inefficient but quick code for finding the postselected samples
a=my_samp
b=a[np.where(a[:,0] == 2)]
c=b[np.where(b[:,2] == 0)]
d=c[np.where(c[:,3] == 1)]
e=d[np.where(d[:,4] == 1)]
print(np.shape(np.array(e)))
print(e)
a=my_samp
b=a[np.where(a[:,0] == 2)]
c=b[np.where(b[:,2] == 1)]
d=c[np.where(c[:,3] == 0)]
e=d[np.where(d[:,4] == 1)]
print(np.shape(np.array(e)))
print(e)
a=my_samp
b=a[np.where(a[:,0] == 2)]
c=b[np.where(b[:,2] == 1)]
d=c[np.where(c[:,3] == 1)]
e=d[np.where(d[:,4] == 0)]
print(np.shape(np.array(e)))
print(e)